### NER with BERT

References:

* https://medium.com/@yingbiao/ner-with-bert-in-action-936ff275bc73

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [0]:
data_path = "/content/drive/My Drive/Datahack NLP Workshop/GMB_NER/"
df = pd.read_csv(data_path + "ner_dataset.csv", encoding = "ISO-8859-1")
df.shape

(1048575, 4)

In [0]:
df = df.fillna(method='ffill')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [0]:
df["Sentence #"].nunique()

47959

In [0]:
df["Word"].nunique()

35178

In [0]:
df["POS"].unique()

array(['NNS', 'IN', 'VBP', 'VBN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'CC',
       'JJ', '.', 'VBD', 'WP', '``', 'CD', 'PRP', 'VBZ', 'POS', 'VBG',
       'RB', ',', 'WRB', 'PRP$', 'MD', 'WDT', 'JJR', ':', 'JJS', 'WP$',
       'RP', 'PDT', 'NNPS', 'EX', 'RBS', 'LRB', 'RRB', '$', 'RBR', ';',
       'UH', 'FW'], dtype=object)

In [0]:
df["Tag"].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [0]:
df["Tag"].value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

### Parse Data

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(df)

# Get sentence data
sentences = [[s[0] for s in sent] for sent in getter.sentences]

# Get pos data
poses = [[s[1] for s in sent] for sent in getter.sentences]

# Get tag labels data
labels = [[s[2] for s in sent] for sent in getter.sentences]

In [0]:
sentences[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

In [0]:
poses[0]

['NNS',
 'IN',
 'NNS',
 'VBP',
 'VBN',
 'IN',
 'NNP',
 'TO',
 'VB',
 'DT',
 'NN',
 'IN',
 'NNP',
 'CC',
 'VB',
 'DT',
 'NN',
 'IN',
 'JJ',
 'NNS',
 'IN',
 'DT',
 'NN',
 '.']

In [0]:
labels[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

### Target Preparation


In [0]:
tags_vals = list(set(df["Tag"].values)) + ['X', '[CLS]', '[SEP]']
tags_vals

['B-geo',
 'B-eve',
 'I-geo',
 'B-tim',
 'I-gpe',
 'I-org',
 'B-org',
 'I-tim',
 'B-gpe',
 'I-per',
 'B-art',
 'I-eve',
 'I-nat',
 'I-art',
 'B-nat',
 'B-per',
 'O',
 'X',
 '[CLS]',
 '[SEP]']

In [0]:
tag2idx={'B-art': 14,
 'B-eve': 16,
 'B-geo': 0,
 'B-gpe': 13,
 'B-nat': 12,
 'B-org': 10,
 'B-per': 4,
 'B-tim': 2,
 'I-art': 5,
 'I-eve': 7,
 'I-geo': 15,
 'I-gpe': 8,
 'I-nat': 11,
 'I-org': 3,
 'I-per': 6,
 'I-tim': 1,
 'X':17,
 'O': 9,
 '[CLS]':18,
 '[SEP]':19}

In [0]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}
tag2name

{0: 'B-geo',
 1: 'I-tim',
 2: 'B-tim',
 3: 'I-org',
 4: 'B-per',
 5: 'I-art',
 6: 'I-per',
 7: 'I-eve',
 8: 'I-gpe',
 9: 'O',
 10: 'B-org',
 11: 'I-nat',
 12: 'B-nat',
 13: 'B-gpe',
 14: 'B-art',
 15: 'I-geo',
 16: 'B-eve',
 17: 'X',
 18: '[CLS]',
 19: '[SEP]'}

### Creating Features

In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 4.9MB/s 
     |████████████████████████████████| 1.0MB 55.3MB/s 
     |████████████████████████████████| 645kB 50.5MB/s 
     |████████████████████████████████| 860kB 15.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=95b8131a44628ac712bdf1e70829a35270fb984de695b5aa3fe1105bc74f75c3
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import torch
from torch import nn
from torch.utils import data

from transformers import (WEIGHTS_NAME, 
                          BertConfig, BertForTokenClassification, BertTokenizer)
from transformers import AdamW, WarmupLinearSchedule

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForTokenClassification, BertTokenizer),
}

model_name = "bert"
pretrained_model_name = "bert-base-cased"
n_classes = len(tags_vals)

config_class, model_class, tokenizer_class = MODEL_CLASSES[model_name]
config = config_class.from_pretrained(pretrained_model_name)
tokenizer = tokenizer_class.from_pretrained(pretrained_model_name, do_lower_case=False)
model = model_class.from_pretrained(pretrained_model_name, num_labels=n_classes)

100%|██████████| 435779157/435779157 [00:14<00:00, 30708045.68B/s]


In [0]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:28
texts:[CLS] Thousands of demons ##tra ##tors have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country . [SEP]
No.0,len:28
lables:[CLS] O O O X X O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O [SEP]
No.1,len:29
texts:[CLS] Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an I ##A ##EA surveillance system begins functioning . [SEP]
No.1,len:29
lables:[CLS] B-gpe O O O O O O O O O O O O O O B-tim O O O B-org X X O O O O O [SEP]
No.2,len:44
texts:[CLS] He ##lic ##op ##ter guns ##hips Saturday pounded militant hide ##outs in the Or ##ak ##zai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South W ##azi ##rist ##an . [SEP]
No.2,len:44
lables:[CLS] O X X X O X B-tim O O O X O O B-geo X X O O O O O B-org O O O O O O O O O O O O O O B-geo I-geo X X X O [SEP]
No.3,len:16
texts:[CLS] They lef

In [0]:
from keras.preprocessing.sequence import pad_sequences
max_len = 50

# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

Using TensorFlow backend.


[  101 26159  1104  8568  4487  5067  1138  9639  1194  1498  1106  5641
  1103  1594  1107  5008  1105  4555  1103 10602  1104  1418  2830  1121
  1115  1583   119   102     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]


In [0]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])

[18  9  9  9 17 17  9  9  9  0  9  9  9  9  9  0  9  9  9  9  9 13  9  9
  9  9  9 19  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9]


In [0]:
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [0]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
print(segment_ids[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
from sklearn.model_selection import train_test_split
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks, tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=0, test_size=0.3)

In [0]:
tr_inputs = torch.tensor(tr_inputs, dtype=torch.long)
val_inputs = torch.tensor(val_inputs, dtype=torch.long)
tr_tags = torch.tensor(tr_tags, dtype=torch.long)
val_tags = torch.tensor(val_tags, dtype=torch.long)
tr_masks = torch.tensor(tr_masks, dtype=torch.long)
val_masks = torch.tensor(val_masks, dtype=torch.long)
tr_segs = torch.tensor(tr_segs, dtype=torch.long)
val_segs = torch.tensor(val_segs, dtype=torch.long)

batch_num = 10

# Only set token embedding, attention embedding, no segment embedding
train_data = data.TensorDataset(tr_inputs, tr_masks, tr_segs, tr_tags)
train_sampler = data.RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_num, drop_last=True)

valid_data = data.TensorDataset(val_inputs, val_masks, val_segs, val_tags)
valid_sampler = data.SequentialSampler(valid_data)
valid_dataloader = data.DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

### Model Building

In [0]:
import math

epochs = 1
max_grad_norm = 1.0

num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs
# Fine tune model all layer parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [0]:
from tqdm import tqdm,trange

n_gpu=1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.train()

print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=b_segs,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 33571
  Batch size = 10
  Num steps = 3358


Epoch: 100%|██████████| 1/1 [05:16<00:00, 316.66s/it]

Train loss: 0.11722836958115163


### Model Evaluation

In [0]:
import torch.nn.functional as F

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, input_segs, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        y_true.append(temp_1)
        y_pred.append(temp_2)


***** Running evaluation *****
  Num examples =14388
  Batch size = 10


In [0]:
actuals = [item for sl in y_true for item in sl]
preds = [item for sl in y_pred for item in sl]

from sklearn import metrics
print("f1 socre: %f"%(metrics.f1_score(actuals, preds, average="macro")))

f1 socre: 0.527142


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
print("Accuracy score: %f"%(metrics.accuracy_score(actuals, preds)))

Accuracy score: 0.969593


In [0]:
print(metrics.classification_report(actuals, preds))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       134
       B-eve       0.59      0.39      0.47        98
       B-geo       0.86      0.90      0.88     11313
       B-gpe       0.96      0.93      0.95      4802
       B-nat       0.45      0.18      0.26        71
       B-org       0.79      0.72      0.75      5997
       B-per       0.83      0.86      0.84      5120
       B-tim       0.93      0.87      0.90      6242
       I-art       0.00      0.00      0.00       100
       I-eve       0.40      0.11      0.18        88
       I-geo       0.86      0.72      0.78      2287
       I-gpe       0.96      0.43      0.60        60
       I-nat       0.00      0.00      0.00        21
       I-org       0.83      0.76      0.79      5095
       I-per       0.81      0.94      0.87      5160
       I-tim       0.89      0.65      0.75      1996
           O       0.99      0.99      0.99    264402
           X       0.00    